# Using Kissat from Julia
Julia already has a package that can install the binaries of Kissat, named `Kissat_jll`.

In [1]:
using Pkg; Pkg.add("Kissat_jll")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
using Kissat_jll

We use the example from [the minisat user guide](https://dwheeler.com/essays/minisat-user-guide.html):

In [37]:
s = "c Here is a comment.
p cnf 5 3
1 -5 4 0
-1 5 3 4 0
-3 -4 0"

"c Here is a comment.\np cnf 5 3\n1 -5 4 0\n-1 5 3 4 0\n-3 -4 0"

Now we write this string to a file in a new temporary directory:

In [38]:
path = mktempdir()
println("Tempdir is $path")
filename = path * "/example.cnf"
open(filename, "w") do file
    write(file, s)
end

Tempdir is /tmp/jl_rbyM3u


58

In [39]:
cmd = Kissat_jll.kissat()

setenv(`/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/bin/kissat`,["EDITOR=/usr/bin/vim", "PATH=/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/bin:/home/lars/ompi/bin:/home/lars/ompi/bin:/usr/lib64/qt-3.3/bin:/usr/share/Modules/bin:/usr/condabin:/usr/lib64/ccache:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/var/lib/snapd/snap/bin:/home/lars/.local/bin:/home/lars/bin", "QTDIR=/usr/lib64/qt-3.3", "LD_LIBRARY_PATH=/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/lib:/home/lars/installed/julia-1.8.5/bin/../lib/julia:/home/lars/installed/julia-1.8.5/bin/../lib::/usr/local/lib:/usr/local/lib", "DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus", "DESKTOP_STARTUP_ID=i3/i3-sensible-terminal/2667-828-nomurai_TIME344518329", "MANPATH=/usr/share/man:", "MODULEPATH=/etc/scl/modulefiles:/etc/scl/modulefiles:/usr/share/Modules/modulefiles:/etc/modulefiles:/usr/share/modulefiles", "XDG_SESSION_DESKTOP=i3", "SSH_AGENT_PID=2717"

In [46]:
out = Pipe()
err = Pipe()
status = run(pipeline(Cmd(`$cmd $filename`, ignorestatus=true), stdout=out, stderr=err))

Process(setenv(`/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/bin/kissat /tmp/jl_rbyM3u/example.cnf`,["EDITOR=/usr/bin/vim", "PATH=/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/bin:/home/lars/ompi/bin:/home/lars/ompi/bin:/usr/lib64/qt-3.3/bin:/usr/share/Modules/bin:/usr/condabin:/usr/lib64/ccache:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/var/lib/snapd/snap/bin:/home/lars/.local/bin:/home/lars/bin", "QTDIR=/usr/lib64/qt-3.3", "LD_LIBRARY_PATH=/home/lars/.julia/artifacts/97c8d6272ffe420241a5ec8f5bf6702987307e29/lib:/home/lars/installed/julia-1.8.5/bin/../lib/julia:/home/lars/installed/julia-1.8.5/bin/../lib::/usr/local/lib:/usr/local/lib", "DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus", "DESKTOP_STARTUP_ID=i3/i3-sensible-terminal/2667-828-nomurai_TIME344518329", "MANPATH=/usr/share/man:", "MODULEPATH=/etc/scl/modulefiles:/etc/scl/modulefiles:/usr/share/Modules/modulefiles:/etc/modulefiles:/usr/share/modulefiles", "XDG_SESSIO

This looks bulky, but apparently sat solvers will give a non-zero exit code even on success and we want to capture this instead of having to catch an error.

In [47]:
status.exitcode

10

In [48]:
close(out.in)
close(err.in)

In [49]:
out.out

Base.PipeEndpoint(RawFD(61) open, 0 bytes waiting)

Print the entire output:

In [50]:
outputlines = collect(eachline(out))

60-element Vector{String}:
 "c ---- [ banner ] ------------------------------------------------------------"
 "c"
 "c Kissat SAT Solver"
 "c "
 "c Copyright (c) 2021-2022 Armin Biere University of Freiburg"
 "c Copyright (c) 2019-2021 Armin Biere Johannes Kepler University Linz"
 "c "
 "c Version 3.0.0 c25429fbce1b5c74d5372e39d681826b33ddaf18"
 "c cc -W -O -DNDEBUG -fpic "
 "c Mon Dec 5 20:39:39 UTC 2022 Linux amdci7 5.4.0-122-generic x86_64"
 "c"
 "c ---- [ parsing ] -----------------------------------------------------------"
 "c"
 ⋮
 "c conflicts:                                0                0.00 per second"
 "c decisions:                                2                0.00 per conflict"
 "c propagations:                             4             1987    per second"
 "c"
 "c ---- [ resources ] ---------------------------------------------------------"
 "c"
 "c maximum-resident-set-size:        549072896 bytes        524 MB"
 "c process-time:                                      

Note that we can only call `eachline` once ont the output pipe. Afterward the result is gone, unless we save it to a variable. Now let us get at the interesting lines, there should be one starting with `s` giving us the status:

In [53]:
status = outputlines[findfirst(line -> line[1] == 's', outputlines)]

"s SATISFIABLE"

In [56]:
if status == "s SATISFIABLE"
    solution = outputlines[findfirst(line -> line[1] == 'v', outputlines)]
    println(solution)
end

v 1 2 3 -4 5 0
